# **Occupancy Detection - by Arya Adhy**

![banner](https://raw.githubusercontent.com/aryaadhy/Occupancy_Detection_FINPRO/main/assets/banner.png)

# Data Preparation 

In [ ]:
import pandas as pd

In [20]:
url_datatest_1 = "https://raw.githubusercontent.com/aryaadhy/Occupancy_Detection_FINPRO/main/dataset/datatest.csv"
url_datatest_2 = "https://raw.githubusercontent.com/aryaadhy/Occupancy_Detection_FINPRO/main/dataset/datatest2.csv"
url_datatraining = "https://raw.githubusercontent.com/aryaadhy/Occupancy_Detection_FINPRO/main/dataset/datatraining.csv"
datatest_1 = pd.read_csv(url_datatraining)
datatest_1.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,2/4/2015 17:51,23.18,27.2720,426.0,721.25,0.004793,1
1,2/4/2015 17:51,23.15,27.2675,429.5,714.00,0.004783,1
2,2/4/2015 17:53,23.15,27.2450,426.0,713.50,0.004779,1
3,2/4/2015 17:54,23.15,27.2000,426.0,708.25,0.004772,1
4,2/4/2015 17:55,23.10,27.2000,426.0,704.50,0.004757,1
